In [1]:
import os
# print(os.getcwd())
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import pickle
import sklearn
print(sklearn.__version__)
import pandas 
import numpy as np
import sys


from evaluation.metric import computeAccuracy, computeF1Score, computeAUC, computeAUCBehavior
# from sklearn.metrics import roc_auc_score

# %load_ext autoreload
# %autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline

0.17


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
# clf_fpa_initial = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
clf_behavior_initial = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pandas.read_csv("./data/alcohol_training_instances.csv")

Prepare the behavior level data

In [3]:
# appending labels based on whether tweet label contains phrase
behavior_labels = []
behavior_index_in_df = []

fpl = "'first_person_level'"

# extracting FP levels and constructing subset vector simultaneously
for i in range(len(df)):
    if fpl in df.labels[i]:
        behavior_labels.append(int(df.labels[i][df.labels[i].find(fpl) + len(fpl) + 2]))
        behavior_index_in_df.append(1)
    else:
        behavior_index_in_df.append(0)

behavior_labels = list(map(int, behavior_labels))
print(behavior_labels[:20])
# print(subset_vec_alc_fp[:10])
print('length of first person level labels: ', len(behavior_labels)) # 6357
print(behavior_index_in_df[:20])
print('length of first person level index in df: ', len(behavior_index_in_df)) # 15651
index_behavior = list(np.where(behavior_index_in_df)[0])
print(index_behavior[:15])
print('length of first person level df index: ', len(index_behavior)) # 6357
df_behavior = df.iloc[index_behavior,]
# df_fp.head
# fp_labels = np.asarray(fp_labels)

# change label 3 to label 0
behavior_labels = [0 if x == 3 else x for x in behavior_labels] 

# train test dataset split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_behavior, behavior_labels, test_size=0.2, random_state=26)

[0, 0, 2, 2, 1, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0]
length of first person level labels:  6357
[0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0]
length of first person level index in df:  15651
[2, 5, 6, 7, 11, 13, 17, 18, 21, 22, 23, 24, 26, 27, 28]
length of first person level df index:  6357


In [4]:
print(len(behavior_labels))
print(len(behavior_index_in_df))
# behavior_labels[:30]
# for i in behavior_labels:
#     if i ==3:
#         print(True)

6357
15651


Now we make our own RF model

In [5]:
# RF
clf_behavior_RF = AlcoholPipeline(global_features=["text"]).pipeline(RandomForestClassifier())
clf_behavior_RF.steps

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
      ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False))]

In [6]:
clf_behavior_RF.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

Predict in testing set

In [8]:
# predict in testing set.
y_behavior_RF = clf_behavior_RF.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_behavior_RF, y_test)
f1_score_RF = computeF1Score(y_test, y_behavior_RF, average = None)
print('F1 Score: ', f1_score_RF)
auc_score_RF = computeAUCBehavior(clf_behavior_RF, X_test, y_test, plot=False, RF=True)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('RF classifier AUC: ', auc_score_RF)

length of total comps:  1272
625 different item in whole dataset
accuracy: 50.864780%
F1 Score:  [0.65712427 0.33       0.21130221]
RF classifier AUC:  {'current': 0.669494989533141, 'looking': 0.6354579278498923, 'reflecting': 0.6112069365592563, 'micro': 0.6943866564811518, 'macro': 0.6387199513140964}


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Test in training set to figure out why the test accuracy is low

In [9]:
# test in training set to figure out why the test accuracy is low
y_train_behavior_RF = clf_behavior_RF.predict(X_train)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_train_behavior_RF, y_train)
f1_train_score_RF = computeF1Score(y_train, y_train_behavior_RF, average = None)
print('F1 Score: ', f1_train_score_RF)
auc_score_train_RF = computeAUCBehavior(clf_behavior_RF, X_train, y_train, plot=False, RF=True)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('RF classifier AUC: ', auc_score_train_RF)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  5085
78 different item in whole dataset
accuracy: 98.466077%
F1 Score:  [0.98697868 0.98435712 0.97955556]
RF classifier AUC:  {'current': 0.9992311070355371, 'looking': 0.9996348294176197, 'reflecting': 0.9994264336379606, 'micro': 0.9993557409969556, 'macro': 0.9994307900303725}


We can see the performance in training set is really good. Overfitting.

Update structure parameters in RF model. To make sure they become same with the initial model

In [14]:
clf_behavior_RF_updateParams = AlcoholPipeline(global_features=["text"]).pipeline(RandomForestClassifier())
params_RF = {'features__text__tfidf__analyzer':'word', 'features__text__tfidf__dtype':np.int64, 
             'features__text__tfidf__max_features':94963, 'features__text__tfidf__ngram_range':(1,3) 
            }
# to see all parameters in model
# clf_alc_LR_updateParams.get_params().keys()
clf_behavior_RF_updateParams.set_params(**params_RF)

Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [15]:
clf_behavior_RF_updateParams.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

Predict in testing set

In [17]:
# predict in testing set.
y_behavior_RF_updateParams = clf_behavior_RF_updateParams.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_behavior_RF_updateParams, y_test)
f1_score_RF_updateParams = computeF1Score(y_test, y_behavior_RF_updateParams, average = None)
print('F1 Score: ', f1_score_RF_updateParams)
auc_score_RF_updateParams = computeAUCBehavior(clf_behavior_RF_updateParams, X_test, y_test, plot=False, RF=True)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('RF classifier AUC: ', auc_score_RF_updateParams)

length of total comps:  1272
490 different item in whole dataset
accuracy: 61.477987%
F1 Score:  [0.72174488 0.52923077 0.33595801]
RF classifier AUC:  {'current': 0.7324014319158688, 'looking': 0.7550268644018161, 'reflecting': 0.7112641740018255, 'micro': 0.7745285182053717, 'macro': 0.7330141438851336}


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
